## A1 - Banco de Dados
`Integrantes: Amanda Perez, Julia Queiroz e Kenner Azevedo`
<br>
`17 de setembro de 2021`

In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass
import warnings
warnings.filterwarnings('ignore')

In [2]:
# dados gerados a partir do Fake Data Generator
df = pd.read_csv('books.csv', delimiter = ',', encoding = 'utf-8')
df.head()

,Unnamed: 0,nome_livro,rating,idioma,qtd_paginas,autor,genero,editora,loja,preco,uf,cliente
0,0,Harry Potter,1,Inglês,477,Oren E. Mckee,Poesia,Metus PC,Estante Virtual,81,RJ,Whilemina Guy
1,1,Livro 3,3,Espanhol,421,Emery X. Allen,Suspense,Purus Limited,Livraria Cultura,58,PB,Karen Erickson
2,2,Livro do Kenner,3,Português,519,Hanna K. Ford,Poesia,Non Ante Bibendum LLC,Estante Virtual,95,PR,Martin Stafford
3,3,Livro Amaldiçoado,3,Espanhol,378,Joseph Cote,Horror,Neque Company,Estante Virtual,49,SC,Ira Gay
4,4,Livro do Renato,4,Francês,561,Ulysses Roberson,Horror,Aliquet Phasellus Industries,Estante Virtual,89,MG,Leilani Benton


In [3]:
# gerando ids

df['id_livro'] = df.index + 1
df['id_compra'] = df.index + 1

col = ['autor', 'genero', 'editora', 'loja', 'uf', 'cliente']
for i in col:
    df['id_{}'.format(i)] = df.groupby([i]).ngroup()
    df['id_{}'.format(i)] = df['id_{}'.format(i)] + 1
    df.rename(columns = {i: 'nome_{}'.format(i)}, inplace = True)

## Criando tabelas

In [4]:
df_autor = df[['id_autor', 'nome_autor']].drop_duplicates(subset = 'id_autor', keep = 'first').sort_values(by = 'id_autor', ascending = True).reset_index(drop = True)

df_genero = df[['id_genero', 'nome_genero']].drop_duplicates(subset = 'id_genero', keep = 'first').sort_values(by = 'id_genero', ascending = True).reset_index(drop = True)

df_editora = df[['id_editora', 'nome_editora']].drop_duplicates(subset = 'id_editora', keep = 'first').sort_values(by = 'id_editora', ascending = True).reset_index(drop = True)

df_uf = df[['id_uf', 'nome_uf']].drop_duplicates(subset = 'id_uf', keep = 'first').sort_values(by = 'id_uf', ascending = True).reset_index(drop = True)

df_cliente = df[['id_cliente', 'nome_cliente', 'id_uf']].drop_duplicates(subset = 'id_cliente', keep = 'first').sort_values(by = 'id_cliente', ascending = True).reset_index(drop = True)
df_cliente.rename(columns = {'id_uf': 'uf_id_uf'}, inplace = True)

df_loja = df[['id_loja', 'nome_loja', 'id_uf']].drop_duplicates(subset = 'id_loja', keep = 'first').sort_values(by = 'id_loja', ascending = True).reset_index(drop = True)
df_loja.rename(columns = {'id_uf': 'uf_id_uf'}, inplace = True)

df_compra = df[['id_compra', 'preco', 'id_cliente', 'id_loja']]
df_compra.rename(columns = {'id_cliente': 'cliente_id_cliente', 'id_loja': 'loja_id_loja'}, inplace = True)

In [5]:
# tabelas auxiliares
df_aux_livro_loja = df[['id_livro', 'id_loja']]
df_aux_livro_loja['id_aux_livro_loja'] = df_aux_livro_loja.index + 1
df_aux_livro_loja.rename(columns = {'id_loja': 'loja_id_loja', 'id_livro': 'livro_id_livro'}, inplace = True)

df_aux_livro_compra = df[['id_livro', 'id_compra']]
df_aux_livro_compra['id_aux_livro_compra'] = df_aux_livro_compra.index + 1
df_aux_livro_compra.rename(columns = {'id_compra': 'compra_id_compra', 'id_livro': 'livro_id_livro'}, inplace = True)

df_aux_autor_livro = df[['id_livro', 'id_autor']]
df_aux_autor_livro['id_aux_autor_livro'] = df_aux_autor_livro.index + 1
df_aux_autor_livro.rename(columns = {'id_autor': 'autor_id_autor', 'id_livro': 'livro_id_livro'}, inplace = True)

df_aux_genero_livro = df[['id_livro', 'id_genero']]
df_aux_genero_livro['id_aux_genero_livro'] = df_aux_genero_livro.index + 1
df_aux_genero_livro.rename(columns = {'id_genero': 'genero_id_genero', 'id_livro': 'livro_id_livro'}, inplace = True)

df_aux_editora_livro = df[['id_livro', 'id_editora']]
df_aux_editora_livro['id_aux_editora_livro'] = df_aux_editora_livro.index + 1
df_aux_editora_livro.rename(columns = {'id_editora': 'editora_id_editora', 'id_livro': 'livro_id_livro'}, inplace = True)

In [6]:
# tabela livros
df_livro = df[['id_livro', 'nome_livro', 'qtd_paginas', 'rating', 'idioma']]

## Conectando à base de dados

In [7]:
p = getpass.getpass()

········


In [8]:
conn = pymysql.connect(host = '127.0.0.1', port = 3306, user = 'root', passwd = p)
cur = conn.cursor()

## Criando tabelas a partir de um arquivo SQL
O arquivo SQL foi extraído do Vertabelo, ambiente onde foi criado o modelo físico.

In [9]:
with open('a1-database_create.sql') as f:
    create_database_sql = f.read()
create_database_sql_list = create_database_sql.split(';')
create_database_sql_list = [q+';' for q in create_database_sql_list][:-1]

for query in create_database_sql_list:
    print(query)
    print(cur.execute(query))

-- Created by Vertabelo (http://vertabelo.com)
-- Last modification date: 2021-09-17 23:54:19.316

-- tables
-- Table: autor
CREATE DATABASE IF NOT EXISTS Livros;
1


USE Livros;
0


CREATE TABLE autor (
    id_autor int NOT NULL,
    nome_autor varchar(35) NOT NULL,
    CONSTRAINT id_autor PRIMARY KEY (id_autor)
);
0


-- Table: aux_autor_livro
CREATE TABLE aux_autor_livro (
    livro_id_livro int NOT NULL,
    autor_id_autor int NOT NULL,
    id_aux_autor_livro int NOT NULL,
    CONSTRAINT aux_autor_livro_pk PRIMARY KEY (id_aux_autor_livro)
);
0


-- Table: aux_editora_livro
CREATE TABLE aux_editora_livro (
    livro_id_livro int NOT NULL,
    editora_id_editora int NOT NULL,
    id_aux_editora_livro int NOT NULL,
    CONSTRAINT aux_editora_livro_pk PRIMARY KEY (id_aux_editora_livro)
);
0


-- Table: aux_genero_livro
CREATE TABLE aux_genero_livro (
    livro_id_livro int NOT NULL,
    genero_id_genero int NOT NULL,
    id_aux_genero_livro int NOT NULL,
    CONSTRAINT aux_genero_livro

## Populando tabelas

In [10]:
dic = {'autor': df_autor, 'editora': df_editora, 'livro': df_livro, 'genero': df_genero, 'uf': df_uf, 'cliente': df_cliente,
       'loja': df_loja, 'compra': df_compra, 'aux_livro_loja': df_aux_livro_loja, 'aux_livro_compra': df_aux_livro_compra,
       'aux_autor_livro': df_aux_autor_livro, 'aux_genero_livro': df_aux_genero_livro, 'aux_editora_livro': df_aux_editora_livro}
for key in dic:
    cols = "`,`".join([str(i) for i in dic[key].columns.tolist()])
    for i, row in dic[key].iterrows():
        sql = f"INSERT INTO `{key}` (`" + cols + "`) VALUES (" + "%s,"*(len(row) - 1) + "%s)"
        cur.execute(sql, tuple(row))
        conn.commit()